In [8]:
!pip3 install boostaroota

In [2]:
import os
import time

import lightgbm as lightgbm
import pandas as pd
import numpy as np
import xgboost as xgboost
from sklearn import model_selection
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, VotingClassifier, AdaBoostClassifier, \
    BaggingClassifier, ExtraTreesClassifier
from sklearn.feature_selection import RFECV
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Lasso, ElasticNet, LogisticRegression, SGDClassifier, LogisticRegressionCV
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, train_test_split
from boruta import BorutaPy
from sklearn.linear_model import LogisticRegression
import pickle
from boostaroota import BoostARoota
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC, NuSVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier

DATA_PATH = '.'
# INPUT_DATA_TRAIN = os.path.join(DATA_PATH, 'input.csv')
# OUTPUT_DATA_TRAIN = os.path.join(DATA_PATH, 'output.csv')

input_df = pickle.load(open( "./full_train.pkl", "rb" ))

train_df = pickle.load(open( "./full_train_10C.pkl", "rb" ))
test_df = pickle.load(open( "./full_test_10C.pkl", "rb" ))


dict_names = {
    'B00300S' : 'Air-temperature',
    'B00305A' :	'Soil-temperature',
    'B00202A' : 'Wind-direction',
    'B00702A' : 'Avg-wind-speed-past-10-mts',
    'B00703A' : 'Max-speed',
    'B00608S' : 'Precipitation-sum-past-10-mts',
    'B00604S' : 'Precipitation-sum-past-24-hrs',
    'B00606S' : 'Precipitation-sum-past-1-hrs',
    'B00802A' : 'Relative-humidity',
    'B00714A' : 'Max-wind-past-10-mts',
    'B00910A' : 'Water-in-snow'
}

def fix_column_names(df, d):
    rename_d = {}
    for column in df.columns:
        column_list = column.split("_")
        station_name = column_list[0]
        coded_name = column_list[1]
        suffix = coded_name
        if coded_name in d:
            suffix = d[coded_name]
        rename_d[station_name+'_'+coded_name] = station_name+"_"+suffix
    df.rename(columns=rename_d,inplace=True)
    return df

def timing(f):
    def wrap(*args):
        time1 = time.time()
        ret = f(*args)
        time2 = time.time()
        print('{:s} function took {:.3f} ms'.format(f.__name__, (time2 - time1) * 1000.0))
        return ret

    return wrap

def prepare_data(data):
    # data = data.drop(columns=['data_'])
    y=None
    if 'is_rain' in data:
        y = data['is_rain']
        X = data.drop(columns=['is_rain'])
    else:
        X = data
    print(X.isnull().values.any())
    X.fillna(method='ffill', inplace=True)
    X.fillna(method='bfill', inplace=True)
    print(X.isnull().values.any())
    X = fix_column_names(X, dict_names)
    return X, y

def prepare_train_test(X,y):
    # y_with_one_indices = y.index[y == 1].tolist()
    # y_with_zero_indices = y.index[y == 0].tolist()
    # y_ones = y[y_with_one_indices]
    # x_ones = X.loc[y_with_one_indices]
    # y_zero = y[y_with_zero_indices][:len(y_ones)]
    # x_zero = X.loc[y_with_zero_indices][:len(y_ones)]
    #
    # X = pd.concat([x_zero, x_ones])
    # y = pd.concat([y_zero, y_ones])

    # idx = np.random.permutation(x_zero.index)
    # x_zero.reindex(idx)
    # y_zero.reindex(idx)
    #
    # x_zero

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, shuffle=False)
    # X_test = X[:140]
    # X_train = X[140:]
    # y_test = y[:140]
    # y_train = y[140:]
    # d_1 = boostaroota(X, y)
    # print('boruta')
    # a_1 = boruta(X.values, y)
    # print('rfecv')
    # b_1 = rfe(X.values, y)
    print('rf')
    # X_train_selected, selected_features = boruta(X_train, y_train)
    X_train_selected, selected_features = rf(X_train, y_train)
    # X_train_selected, selected_features = rfe(X_train, y_train)
    X_test_selected = X_test[selected_features]

    x_standard_scaler = StandardScaler()
    X_train_selected_transformed = x_standard_scaler.fit_transform(X_train_selected)
    X_test_selected_transformed = x_standard_scaler.transform(X_test_selected)

    y_label_encoder = LabelEncoder()
    y_train_encoded = y_label_encoder.fit_transform(y_train)
    y_test_encoded = y_label_encoder.transform(y_test)

    return X_train_selected_transformed, X_test_selected_transformed, y_train_encoded, y_test_encoded


# def divide_train_test(X,y):
def boostaroota(df_x, df_y):
    br = BoostARoota(metric='f1')
    br.fit(df_x, df_y)
    df_x_selected = br.fit_transform(df_x, df_y)
    return df_x_selected

@timing
def boruta(df_x, df_y):
    columns = list(df_x.columns)
    df_x = df_x.values
    rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)
    feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1)
    df_x_selected = feat_selector.fit_transform(df_x, df_y)
    selected_cols = []
    for (f,i) in zip(columns, feat_selector.support_):
        if i == True:
            selected_cols.append(f)
    return df_x_selected, selected_cols#, list(df_x_selected.columns)

@timing
def rfe(df_x, df_y):
    columns = list(df_x.columns)
    rfc = RandomForestClassifier(random_state=101)
    rfecv = RFECV(estimator=rfc, step=3,
                     cv=StratifiedKFold(2), scoring='f1_weighted')
    # rfecv = RFECV(estimator=rfc, step=1, cv=StratifiedKFold(2), scoring="f1_weighted")
    df_x_selected = rfecv.fit_transform(df_x, df_y)
    print(df_x_selected.shape)
    selected_cols = []
    for (f, i) in zip(columns, rfecv.support_):
        if i == True:
            selected_cols.append(f)
    return df_x_selected, selected_cols #, list(df_x_selected.columns)

@timing
def rf(df_x, df_y):
    rf = RandomForestClassifier(n_jobs=-1, random_state=1, n_estimators=20)
    rf.fit(df_x, df_y)
    names = df_x.columns
    feat_imp = rf.feature_importances_
    print ("Features sorted by their score:")
    print(sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), names),
           reverse=True))
    print(feat_imp)
    feats_selected = df_x.columns[feat_imp > 0.01]
    # print(df_x_selected.shape)
    return df_x[feats_selected], feats_selected #, list(df_x_selected.columns)

# class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
#     def __init__(self, models):
#         self.models_ = models
#
#     def fit(self, X, y):
#         for model in self.models_:
#             model.fit(X, y)
#
#     def predict(self, X):
#         predictions = np.column_stack([
#             model.predict(X) for model in self.models_
#         ])
#         return np.mean(predictions, axis=1)

class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        # self.models_ = models
        self.voting_classifier = VotingClassifier(estimators=[models], voting='hard')

    def fit(self, X, y):
        self.voting_classifier.fit(X, y)

    def predict(self, X):
        return self.voting_classifier.predict(X)

    # > eclf3 = VotingClassifier(estimators=[
    #     ...('lr', clf1), ('rf', clf2), ('gnb', clf3)],
    #     ...
    # voting = 'soft', weights = [2, 1, 1],
    #                            ...
    # flatten_transform = True)
    # >> > eclf3 = eclf3.fit(X, y)
    # >> > print(eclf3.predict(X))

def return_models():
        models = []

        ### random forest model
        models.append( ('rf', RandomForestClassifier(max_depth=5, random_state=42, n_estimators=5,
                                         verbose=0) ) ) # Train the model on training data
#         models.append( ('knn', KNeighborsClassifier(3)) )

#         models.append(('svc',SVC(kernel="linear", C=0.025)))
#         models.append(('dt', DecisionTreeClassifier(max_depth=5)))
#         models.append(('ada', AdaBoostClassifier()))
#         models.append(('gaussian', GaussianNB()))
#         # models.append(('nusvc', NuSVC()))
#         models.append(('linear_svc', LinearSVC(max_iter=10000)))
#         models.append(('sgd_classifier', SGDClassifier()))
#         models.append(('logisit_reg',LogisticRegressionCV(max_iter=10000)))
#         models.append(('bagging_classifier', BaggingClassifier()))
#         models.append(('extra_trees', ExtraTreesClassifier()))

        return models


#####################

# X = pd.read_csv('./examples/test_X', index_col=0)
# y = pd.read_csv('./examples/test_y', header=None, index_col=0)
# y = y.ravel()

X_train, y_train = prepare_data(train_df)
X_test, y_test = prepare_data(test_df)

x_standard_scaler = StandardScaler()
X_train_selected_transformed = x_standard_scaler.fit_transform(X_train)
X_test_selected_transformed = x_standard_scaler.transform(X_test)

y_label_encoder = LabelEncoder()
y_train_encoded = y_label_encoder.fit_transform(y_train)
# y_test_encoded = y_label_encoder.transform(y_test)

# X_train, X_test, y_train, y_test = prepare_train_test(X,y)
models = return_models()
voting_classifier = VotingClassifier(estimators=models, voting='hard')
voting_classifier.fit(X_train, y_train)
cv = model_selection.cross_val_score(voting_classifier, X_train, y_train,cv=10, scoring='f1')
print(cv)
print(np.mean(cv))
y_pred = voting_classifier.predict(X_test)

pickle.dump(y_pred,open("result.obj","wb"))
print('f1 score {}'.format(f1_score(y_test, y_pred, zero_division=1)))

True
False
True
False


/home/gregory_spaxwell/venv/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/gregory_spaxwell/venv/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/gregory_spaxwell/venv/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_i

[0.16666667 0.46666667 0.         0.18181818 0.         0.
 0.         0.31578947 1.         0.57142857]
0.2702369560264297


ValueError: Expected array-like (array or non-string sequence), got None

In [84]:

def prepare_data(data):
    # data = data.drop(columns=['data_'])
    y=None
    if 'is_rain' in data:
        y = data['is_rain']
        X = data.drop(columns=['is_rain'])
    else:
        X = data
    print(X.isnull().values.any())
    X.fillna(method='ffill', inplace=True)
    X.fillna(method='bfill', inplace=True)
    print(X.isnull().values.any())
    X = fix_column_names(X, dict_names)
    return X, y

def prepare_train_test(X,y):
    # y_with_one_indices = y.index[y == 1].tolist()
    # y_with_zero_indices = y.index[y == 0].tolist()
    # y_ones = y[y_with_one_indices]
    # x_ones = X.loc[y_with_one_indices]
    # y_zero = y[y_with_zero_indices][:len(y_ones)]
    # x_zero = X.loc[y_with_zero_indices][:len(y_ones)]
    #
    # X = pd.concat([x_zero, x_ones])
    # y = pd.concat([y_zero, y_ones])

    # idx = np.random.permutation(x_zero.index)
    # x_zero.reindex(idx)
    # y_zero.reindex(idx)
    #
    # x_zero

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, shuffle=False)
    # X_test = X[:140]
    # X_train = X[140:]
    # y_test = y[:140]
    # y_train = y[140:]
    # d_1 = boostaroota(X, y)
    # print('boruta')
    # a_1 = boruta(X.values, y)
    # print('rfecv')
    # b_1 = rfe(X.values, y)
    print('rf')
    # X_train_selected, selected_features = boruta(X_train, y_train)
    X_train_selected, selected_features = rf(X_train, y_train)
    # X_train_selected, selected_features = rfe(X_train, y_train)
    X_test_selected = X_test[selected_features]

    x_standard_scaler = StandardScaler()
    X_train_selected_transformed = x_standard_scaler.fit_transform(X_train_selected)
    X_test_selected_transformed = x_standard_scaler.transform(X_test_selected)

    y_label_encoder = LabelEncoder()
    y_train_encoded = y_label_encoder.fit_transform(y_train)
    y_test_encoded = y_label_encoder.transform(y_test)

    return X_train_selected_transformed, X_test_selected_transformed, y_train_encoded, y_test_encoded


# def divide_train_test(X,y):
def boostaroota(df_x, df_y):
    br = BoostARoota(metric='f1')
    br.fit(df_x, df_y)
    df_x_selected = br.fit_transform(df_x, df_y)
    return df_x_selected

@timing
def boruta(df_x, df_y):
    columns = list(df_x.columns)
    df_x = df_x.values
    rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)
    feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1)
    df_x_selected = feat_selector.fit_transform(df_x, df_y)
    selected_cols = []
    for (f,i) in zip(columns, feat_selector.support_):
        if i == True:
            selected_cols.append(f)
    return df_x_selected, selected_cols#, list(df_x_selected.columns)

@timing
def rfe(df_x, df_y):
    columns = list(df_x.columns)
    rfc = RandomForestClassifier(random_state=101)
    rfecv = RFECV(estimator=rfc, step=3,
                     cv=StratifiedKFold(2), scoring='f1_weighted')
    # rfecv = RFECV(estimator=rfc, step=1, cv=StratifiedKFold(2), scoring="f1_weighted")
    df_x_selected = rfecv.fit_transform(df_x, df_y)
    print(df_x_selected.shape)
    selected_cols = []
    for (f, i) in zip(columns, rfecv.support_):
        if i == True:
            selected_cols.append(f)
    return df_x_selected, selected_cols #, list(df_x_selected.columns)

@timing
def rf(df_x, df_y):
    rf = RandomForestClassifier(n_jobs=-1, random_state=1, n_estimators=20)
    rf.fit(df_x, df_y)
    names = df_x.columns
    feat_imp = rf.feature_importances_
    print ("Features sorted by their score:")
    print(sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), names),
           reverse=True))
    print(feat_imp)
    feats_selected = df_x.columns[feat_imp > 0.01]
    # print(df_x_selected.shape)
    return df_x[feats_selected], feats_selected #, list(df_x_selected.columns)

# class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
#     def __init__(self, models):
#         self.models_ = models
#
#     def fit(self, X, y):
#         for model in self.models_:
#             model.fit(X, y)
#
#     def predict(self, X):
#         predictions = np.column_stack([
#             model.predict(X) for model in self.models_
#         ])
#         return np.mean(predictions, axis=1)

class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        # self.models_ = models
        self.voting_classifier = VotingClassifier(estimators=[models], voting='hard')

    def fit(self, X, y):
        self.voting_classifier.fit(X, y)

    def predict(self, X):
        return self.voting_classifier.predict(X)

    # > eclf3 = VotingClassifier(estimators=[
    #     ...('lr', clf1), ('rf', clf2), ('gnb', clf3)],
    #     ...
    # voting = 'soft', weights = [2, 1, 1],
    #                            ...
    # flatten_transform = True)
    # >> > eclf3 = eclf3.fit(X, y)
    # >> > print(eclf3.predict(X))

def return_models():
        models = []

        ### random forest model
#         models.append( ('rf', RandomForestClassifier(max_depth=5, random_state=42, n_estimators=5,
#                                          verbose=0) ) ) # Train the model on training data
#         models.append( ('knn', KNeighborsClassifier(3)) )

#         models.append(('svc',SVC(kernel="linear", C=0.025)))
#         models.append(('dt', DecisionTreeClassifier(max_depth=5)))
#         models.append(('ada', AdaBoostClassifier()))
#         models.append(('gaussian', GaussianNB()))
#         # models.append(('nusvc', NuSVC()))
#         models.append(('linear_svc', LinearSVC(max_iter=100000)))
#         models.append(('sgd_classifier', SGDClassifier()))
        models.append(('logisit_reg',LogisticRegressionCV(max_iter=100000)))
#         models.append(('bagging_classifier', BaggingClassifier()))
#         models.append(('extra_trees', ExtraTreesClassifier()))

        return models

In [87]:
X_train = pickle.load(open('train_x.pkl','rb'))
y_train = pickle.load(open('train_y.pkl','rb'))


In [74]:
X_test = pickle.load(open('test_x.pkl','rb'))#, y_test = prepare_data(test_df)



In [78]:
X_test.shape

(504, 66)

In [86]:

x_standard_scaler = StandardScaler()
X_train_selected_transformed = x_standard_scaler.fit_transform(X_train)
X_test_selected_transformed = x_standard_scaler.transform(X_test)

y_label_encoder = LabelEncoder()
y_train_encoded = y_label_encoder.fit_transform(y_train)
# y_test_encoded = y_label_encoder.transform(y_test)

# X_train, X_test, y_train, y_test = prepare_train_test(X,y)
models = return_models()
voting_classifier = VotingClassifier(estimators=models, voting='hard')
voting_classifier.fit(X_train, y_train)
# cv = model_selection.cross_val_score(voting_classifier, X_train, y_train,cv=10, scoring='f1')
# print(cv)
# print(np.mean(cv))
y_pred = voting_classifier.predict(X_test)

pickle.dump(y_pred,open("result.obj","wb"))
#print('f1 score {}'.format(f1_score(y_test, y_pred, zero_division=1)))

/home/gregory_spaxwell/venv/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/gregory_spaxwell/venv/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/s

In [13]:
Y_PRED = X_test.copy()
Y_PRED['pred'] = y_pred

In [21]:
Y_PRED[['pred']].to_csv('output.csv',header=False,sep=';')

In [88]:
Y_PRED[Y_PRED[['pred']].index.str[-2:]=='00'][['pred']].to_csv('output.csv',header=False,sep=';')

In [52]:
y_train.shape


(1703,)

In [94]:
y_train.value_counts()

0    1600
1     103
Name: is_rain, dtype: int64